In [1]:
#导包
from netCDF4 import Dataset
from tensorflow.keras import layers, models
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, LayerNormalization,Input, Conv3D, TimeDistributed, Flatten
from tensorflow.keras.layers import Concatenate,Conv2D,TimeDistributed, MaxPooling2D, Input, MaxPooling3D
from tensorflow.keras.layers import  Reshape,multiply
from tensorflow.keras.layers import Layer,Lambda,Dot,ReLU,LSTM, Dense, Dropout, Activation, Flatten,Attention
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import r2_score,accuracy_score,precision_score
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import regularizers
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow.keras as keras
from tensorflow.keras import layers, Sequential, regularizers
# calculate RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd 
import tensorflow as tf
import gc 
import gzip
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import match
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os 
%matplotlib inline

In [2]:
# 检查可用GPU数量
len(tf.config.experimental.list_physical_devices('GPU'))

1

# 读取数据

## SSH

In [3]:
data_ssh = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSH/SSH_2005-2015_month.nc')

In [4]:
# 确认时间
time_ssh = data_ssh['time'][:]
print(len(time_ssh))#查看时间长度为132

132


In [5]:
# 查看研究区域经纬度
ssh_lat = data_ssh['lat'][:].data
#print(ssh_lat[:])
ssh_lon = data_ssh['lon'][:].data
#print(ssh_lon[:])

In [6]:
# 读取ssh
ssh = data_ssh['ssh'][:].data
print(ssh.shape)

(132, 280, 600)


In [7]:
# 查看ssh最大值最小值，看是否有异常值
np.nanmax(ssh),np.nanmin(ssh)

(2.20051, -0.43673334)

## SSS

In [8]:
data_sss = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSS/dataset-sss-ssd-rep-monthly_2005-2020.nc')

In [9]:
# 确定时间
time_sss = data_sss['time'][:132]
print(len(time_sss))

132


In [10]:
# 获取研究位置经纬度
sss_lat = data_sss['lat'][:].data
#print(sss_lat[:])
sss_lon = data_sss['lon'][:].data
#print(sss_lon[:])

In [11]:
sss = data_sss['sos'][:132].data 
print(sss.shape)
sss = np.squeeze(sss) # 移除大小为一的维度
print(sss.shape)

(132, 1, 280, 600)
(132, 280, 600)


In [12]:
np.nanmax(sss),np.nanmin(sss)

(36.072178, 13.604756)

## SSW

In [13]:
data_ssw = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSW/anon_ssw_2005_2018_Month_data.nc')

In [14]:
# 确定时间
time_ssw = data_ssw['time'][:132]
print(len(time_ssw))

132


In [15]:
lat_ssw = data_ssw['lat'][:].data
#print(lat_ssw[314:594])
lon_ssw = data_ssw['lon'][:].data
#print(lon_ssw[440:1040])

# lat 0.125    ----    69.875
# lon  110.125    ----    259.875

In [16]:
# 获取U分量 和 V分量     time   lat    lon
uwnd = data_ssw['uwnd'][:132,314:594,440:1040].data  
vwnd = data_ssw['vwnd'][:132,314:594,440:1040].data

In [17]:
print(uwnd.shape)
print(vwnd.shape)

(132, 280, 600)
(132, 280, 600)


In [18]:
np.nanmax(uwnd),np.nanmin(uwnd),np.nanmax(vwnd),np.nanmin(vwnd)

(12.433594, -11.7988205, 8.8359995, -13.645481)

## 3DT

In [19]:
data_3dt_1 = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/dataset-armor-3d-rep-monthly_2005-2009.nc')
data_3dt_2 = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/dataset-armor-3d-rep-monthly_2010-2015.nc')

In [20]:
sst_1 = data_3dt_1['to'][:].data
sst_2 = data_3dt_2['to'][:].data

In [21]:
print(sst_1.shape)
print(sst_2.shape)

(60, 43, 280, 600)
(72, 43, 280, 600)


###  0 - 2000m 温度数据

In [22]:
# 将sst_1 和 sst_2 合并
temp_depth = np.append(sst_1,sst_2,axis=0)
print(temp_depth.shape)

(132, 43, 280, 600)


In [23]:
# 所有深度层
depths = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
         450,500,550,600,700,800,900,1000,1100,1200,1300,1400,1500,1750,2000]

#选取的深度层
depths_use = [10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
              500,600,700,800,900,1000,1100,1300,1500,1750,2000]

#[50,100,125,150,200,250,300,400,500,600,700,800,900,1000]

In [24]:
temp_depth_use = temp_depth[:,1:2]

In [25]:
# 获取选取的深度层的所有信息
for i,depth in enumerate(depths):
    if(depth in depths_use):
        #print(depths[i])
        temp_depth_use = np.concatenate((temp_depth_use,temp_depth[:,i:i+1]),axis=1)    

In [26]:
print(temp_depth_use.shape)
# 将深度维度放到最后
temp_depth_use = np.transpose(temp_depth_use, (0,2,3,1))
print(temp_depth_use.shape)

(132, 33, 280, 600)
(132, 280, 600, 33)


## 表面温度数据

In [27]:
sst = temp_depth[:,0:1,:,:]

In [28]:
sst = np.squeeze(sst)
print(sst.shape)

(132, 280, 600)


## 数据合并

In [29]:
# 将所有表层数据在通道维度进行拼接
print(ssh.shape,sss.shape,uwnd.shape,vwnd.shape,sst.shape)
cat_data =np.stack((ssh,sss,uwnd,vwnd,sst),axis=3)
cat_data.shape

(132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600)


(132, 280, 600, 5)

In [30]:
# 将表面数据和水下数据在最后一个维度拼接，将空值删除。
cat_data_ss_depth = np.concatenate((cat_data,temp_depth_use),axis=-1)

In [31]:
cat_data_ss_depth.shape   # 5+33=38

(132, 280, 600, 38)

## 将两个温度异常区域的数据设置为nan

In [32]:
test = temp_depth_use[:,:,:,27:28]
test[test < 0.4] = np.nan
test[test > 10] = np.nan

In [33]:
cat_data_ss_depth[:,:,:,32:33] = test

# 验证集和测试集的划分

In [34]:
# 最后一年的数据作为test
test_num = 120
train = cat_data_ss_depth[:test_num]
test = cat_data_ss_depth[test_num:]

In [35]:
train.shape,test.shape

((120, 280, 600, 38), (12, 280, 600, 38))

## 去掉合并数据中的nan


In [36]:
train.shape 

(120, 280, 600, 38)

In [37]:
# 将训练数据展平，因为简单的全连接只涉及数据与数据之间的关系，没涉及到时间和空间。当前就相当于使用便面数据的5个数去反演深度数据的1个数。
train_flatten = train.reshape(-1,38)
#将默认填充值设置为nan
train_flatten[train_flatten == 32767] = np.nan
print('train删除nan值之前',train_flatten.shape)
# 使用 numpy.isnan 函数检测NaN值
nan_mask = np.isnan(train_flatten)
# 通过布尔索引删除NaN值
train_without_nan = train_flatten[~nan_mask.any(axis=1)]
print('train删除nan值之后',train_without_nan.shape)

train删除nan值之前 (20160000, 38)
train删除nan值之后 (12427525, 38)


In [38]:
test_flatten =  test.reshape(-1,38)
test_flatten[test_flatten == 32767] = np.nan
print('test删除nan值之前',test_flatten.shape)
# 使用 numpy.isnan 函数检测NaN值
nan_mask = np.isnan(test_flatten)
# 通过布尔索引删除NaN值
test_without_nan = test_flatten[~nan_mask.any(axis=1)]
print('test删除nan值之后',test_without_nan.shape)

test删除nan值之前 (2016000, 38)
test删除nan值之后 (1242592, 38)


## 划分训练数据和目标数据

In [39]:
# 将表面数据和次表层数据分开
x_test = test_without_nan[:,:5]
y_test = test_without_nan[:,5:]

x_train_val = train_without_nan[:,:5]
y_train_val = train_without_nan[:,5:]

In [40]:
print('x_test.shape:',x_test.shape)
print('y_test.shape',y_test.shape)
print('x_train_val',x_train_val.shape)
print('y_train_val',y_train_val.shape)

x_test.shape: (1242592, 5)
y_test.shape (1242592, 33)
x_train_val (12427525, 5)
y_train_val (12427525, 33)


In [41]:
#划分验证集 和 测试集
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=10)

In [42]:
#查看其形状
x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape

((9320643, 5),
 (1242592, 5),
 (3106882, 5),
 (9320643, 33),
 (1242592, 33),
 (3106882, 33))

# 特征归一化

In [43]:
# 记录数据集的形状
# x_train_shape = x_train.shape
# y_train_shape = y_train.shape

# x_test_shape = x_test.shape
# y_test_shape = y_test.shape

# x_val_shape = x_val.shape
# y_val_shape = y_val.shape

In [44]:
# 归一化的作用：消除不同特征之间的量纲影响。
scaler_f = StandardScaler()
x_train = scaler_f.fit_transform(x_train) 
x_test = scaler_f.transform(x_test)
x_val = scaler_f.transform(x_val)

scaler_l = StandardScaler()
y_train = scaler_l.fit_transform(y_train)
y_test = scaler_l.transform(y_test)
y_val = scaler_l.transform(y_val)

In [45]:
# x_train = np.reshape(x_train,(x_train_shape))
# x_test = np.reshape(x_test,(x_test_shape))
# x_val = np.reshape(x_val,(x_val_shape))
# y_train = np.reshape(y_train,(y_train_shape))
# y_test = np.reshape(y_test,(y_test_shape))
# y_val = np.reshape(y_val,(y_val_shape))

In [46]:
# 特征归一化后没有改变数据形状
x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape

((9320643, 5),
 (1242592, 5),
 (3106882, 5),
 (9320643, 33),
 (1242592, 33),
 (3106882, 33))

# 预聚类（不用管）

In [47]:
# 一下部分为测试预聚类方法， 没有得到较好的结果，先不使用。

In [48]:
# import numpy as np
# from sklearn.cluster import KMeans

# def run_kmeans(data, n_clusters, n_runs):
#     best_score = None
#     best_labels = None
    
#     for _ in range(n_runs):
#         kmeans = KMeans(n_clusters=n_clusters)
#         labels = kmeans.fit_predict(data)
#         score = kmeans.score(data)
        
#         if best_score is None or score < best_score:
#             best_score = score
#             best_labels = labels
    
#     return best_labels

In [49]:
# n_clusters = 28
# n_runs = 20
# labels = run_kmeans(x_train, n_clusters, n_runs)

# print("聚类结果：")
# print(labels)

In [50]:
# #np.savetxt('labels.txt',labels)
# b =  np.loadtxt("labels.txt", delimiter=',')  # 读取文件

## 根据聚类结果排序

In [51]:
# # 将数据和对应的聚类标签合并为一个数组
# data_with_labels = np.column_stack((x_train, labels))
# # 按照聚类标签对数据进行排序
# sorted_data = data_with_labels[data_with_labels[:, -1].argsort()]
# sorted_data_x = sorted_data[:,:5]

In [52]:
# # 将数据和对应的聚类标签合并为一个数组
# data_with_labels = np.column_stack((y_train, labels))
# # 按照聚类标签对数据进行排序
# sorted_data = data_with_labels[data_with_labels[:, -1].argsort()]
# sorted_data_y = sorted_data[:,:15]

In [53]:
# sorted_data_x.shape,sorted_data_y.shape 

# 定义网络结构

In [54]:
# 创建神经网络模型
def create_model():   
    model = models.Sequential([
        layers.Dense(200, activation='tanh', input_shape=(5,)),  # 输入层和隐藏层
        layers.Dense(15, activation='tanh'),
        layers.Dense(8, activation='tanh'),
        layers.Dense(1, activation='linear')  # 输出层，num_depths表示深度的数量
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model 
#model.summary()


# 若自定义损失函数


# # 定义自定义损失函数
# def my_custom_loss(y_true, y_pred):  # 参数位置不能随意交换，第一个参数必须是真实值，第二个参数必须是预测值
#     # 例如，你可以计算平方误差并添加一些自定义的正则化或其他惩罚
#     loss = tf.reduce_mean(tf.square(y_true - y_pred))  # 计算均方误差
#     return loss

# # 在编译模型时使用自定义损失函数
# model.compile(optimizer='adam', loss=my_custom_loss)

## 网格搜索交叉验证寻找最佳参数

In [55]:
# model = KerasRegressor(build_fn=create_model,verbose=2)

In [56]:
# param_grid = {
#     'neurons': [4,8,15,30,50,100,150,200],
# }

In [57]:
# # 使用GridSearchCV进行交叉验证并搜索最佳参数组合：
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
# grid_result = grid_search.fit(x_train, y_train[:,14:15],epochs=200,verbose=2,batch_size=1024)

In [58]:
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

## 编译模型

In [60]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 15)                3015      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 128       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 4,352
Trainable params: 4,352
Non-trainable params: 0
_________________________________________________________________


In [61]:
# # 创建神经网络模型
# num_depths = 1
# num_features = 5
# # # 编译模型
# # model.compile(optimizer='adam', loss='mean_squared_error')
# checkpoint_path='D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/BPNN/temp/bpnn_model.h5'
# keras_callbacks   = [
#       EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
#       ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
# ]

## 模型训练

### 对指定层进行训练

In [62]:
# num_epochs = 1000
# batch_size = 1024

# history = model.fit(sorted_data_x, sorted_data_y[:,14:], validation_data=(x_val,y_val[:,14:]) ,
#                     epochs=num_epochs, batch_size=batch_size, verbose=2, callbacks=keras_callbacks)

### 对所有层进行训练

In [63]:
def BPNN_model(x_train,y_train,x_val,y_val,name):
    # 创建网络
    model = create_model()
    # 编译网络
    model.compile(optimizer='adam', loss='mean_squared_error')
    # 设置早停 和 模型保存路径
    checkpoint_path=name
    keras_callbacks   = [
          EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
          ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
    ]
    # 模型训练
    model.fit(x_train, y_train, validation_data=(x_val,y_val),
        epochs=1000, batch_size=10240, verbose=2, callbacks=keras_callbacks)

In [68]:
#对所有层进行建模
depth=[5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,1300,1500,1750,2000]
for i in range(0,33): #0-14
    BPNN_model(x_train, y_train[:,i:i+1],x_val, y_val[:,i:i+1],
                    'D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/BPNN/temp/model1/bpnn_temp_'+str(depth[i])+'m.h5')

Epoch 1/1000
911/911 - 2s - loss: 0.0174 - val_loss: 0.0015
Epoch 2/1000
911/911 - 1s - loss: 0.0013 - val_loss: 0.0012
Epoch 3/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 4/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 5/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 6/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 7/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 8/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 9/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 10/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 11/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 12/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 13/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 14/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 15/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 16/1000
911/911 - 1s - loss: 0.0012 - val_loss: 0.0012
Epoch 17/1000
911/911 - 1s - loss

911/911 - 1s - loss: 0.0079 - val_loss: 0.0079
Epoch 13/1000
911/911 - 1s - loss: 0.0078 - val_loss: 0.0078
Epoch 14/1000
911/911 - 1s - loss: 0.0078 - val_loss: 0.0077
Epoch 15/1000
911/911 - 1s - loss: 0.0077 - val_loss: 0.0077
Epoch 16/1000
911/911 - 1s - loss: 0.0077 - val_loss: 0.0078
Epoch 17/1000
911/911 - 1s - loss: 0.0077 - val_loss: 0.0077
Epoch 18/1000
911/911 - 1s - loss: 0.0076 - val_loss: 0.0077
Epoch 19/1000
911/911 - 1s - loss: 0.0076 - val_loss: 0.0076
Epoch 20/1000
911/911 - 1s - loss: 0.0076 - val_loss: 0.0075
Epoch 21/1000
911/911 - 1s - loss: 0.0075 - val_loss: 0.0076
Epoch 22/1000
911/911 - 1s - loss: 0.0075 - val_loss: 0.0075
Epoch 23/1000
911/911 - 1s - loss: 0.0075 - val_loss: 0.0075
Epoch 24/1000
911/911 - 1s - loss: 0.0075 - val_loss: 0.0075
Epoch 25/1000
911/911 - 1s - loss: 0.0075 - val_loss: 0.0075
Epoch 26/1000
911/911 - 1s - loss: 0.0074 - val_loss: 0.0074
Epoch 27/1000
911/911 - 1s - loss: 0.0074 - val_loss: 0.0075
Epoch 28/1000
911/911 - 1s - loss: 0.0

Epoch 30/1000
911/911 - 1s - loss: 0.0152 - val_loss: 0.0152
Epoch 31/1000
911/911 - 1s - loss: 0.0152 - val_loss: 0.0154
Epoch 32/1000
911/911 - 1s - loss: 0.0151 - val_loss: 0.0152
Epoch 33/1000
911/911 - 1s - loss: 0.0151 - val_loss: 0.0151
Epoch 34/1000
911/911 - 1s - loss: 0.0151 - val_loss: 0.0151
Epoch 35/1000
911/911 - 1s - loss: 0.0151 - val_loss: 0.0151
Epoch 36/1000
911/911 - 1s - loss: 0.0150 - val_loss: 0.0150
Epoch 37/1000
911/911 - 1s - loss: 0.0150 - val_loss: 0.0151
Epoch 38/1000
911/911 - 1s - loss: 0.0150 - val_loss: 0.0150
Epoch 39/1000
911/911 - 1s - loss: 0.0150 - val_loss: 0.0150
Epoch 40/1000
911/911 - 1s - loss: 0.0149 - val_loss: 0.0150
Epoch 41/1000
911/911 - 1s - loss: 0.0149 - val_loss: 0.0149
Epoch 42/1000
911/911 - 1s - loss: 0.0149 - val_loss: 0.0149
Epoch 43/1000
911/911 - 1s - loss: 0.0149 - val_loss: 0.0149
Epoch 44/1000
911/911 - 1s - loss: 0.0148 - val_loss: 0.0148
Epoch 45/1000
911/911 - 1s - loss: 0.0148 - val_loss: 0.0149
Epoch 46/1000
911/911 - 

Epoch 48/1000
911/911 - 1s - loss: 0.0197 - val_loss: 0.0197
Epoch 49/1000
911/911 - 1s - loss: 0.0197 - val_loss: 0.0197
Epoch 50/1000
911/911 - 1s - loss: 0.0197 - val_loss: 0.0198
Epoch 51/1000
911/911 - 1s - loss: 0.0196 - val_loss: 0.0197
Epoch 52/1000
911/911 - 1s - loss: 0.0196 - val_loss: 0.0196
Epoch 53/1000
911/911 - 1s - loss: 0.0196 - val_loss: 0.0196
Epoch 54/1000
911/911 - 1s - loss: 0.0196 - val_loss: 0.0196
Epoch 55/1000
911/911 - 1s - loss: 0.0195 - val_loss: 0.0196
Epoch 56/1000
911/911 - 1s - loss: 0.0195 - val_loss: 0.0195
Epoch 57/1000
911/911 - 1s - loss: 0.0195 - val_loss: 0.0196
Epoch 58/1000
911/911 - 1s - loss: 0.0195 - val_loss: 0.0195
Epoch 59/1000
911/911 - 1s - loss: 0.0195 - val_loss: 0.0199
Epoch 60/1000
911/911 - 1s - loss: 0.0194 - val_loss: 0.0195
Epoch 61/1000
911/911 - 1s - loss: 0.0194 - val_loss: 0.0195
Epoch 62/1000
911/911 - 1s - loss: 0.0194 - val_loss: 0.0196
Epoch 63/1000
911/911 - 1s - loss: 0.0194 - val_loss: 0.0195
Epoch 64/1000
911/911 - 

Epoch 46/1000
911/911 - 1s - loss: 0.0232 - val_loss: 0.0233
Epoch 47/1000
911/911 - 1s - loss: 0.0232 - val_loss: 0.0232
Epoch 48/1000
911/911 - 1s - loss: 0.0232 - val_loss: 0.0232
Epoch 49/1000
911/911 - 1s - loss: 0.0232 - val_loss: 0.0233
Epoch 50/1000
911/911 - 1s - loss: 0.0231 - val_loss: 0.0232
Epoch 51/1000
911/911 - 1s - loss: 0.0231 - val_loss: 0.0232
Epoch 52/1000
911/911 - 1s - loss: 0.0231 - val_loss: 0.0231
Epoch 53/1000
911/911 - 1s - loss: 0.0231 - val_loss: 0.0232
Epoch 54/1000
911/911 - 1s - loss: 0.0231 - val_loss: 0.0231
Epoch 55/1000
911/911 - 1s - loss: 0.0230 - val_loss: 0.0231
Epoch 56/1000
911/911 - 1s - loss: 0.0230 - val_loss: 0.0231
Epoch 57/1000
911/911 - 1s - loss: 0.0230 - val_loss: 0.0231
Epoch 58/1000
911/911 - 1s - loss: 0.0230 - val_loss: 0.0231
Epoch 59/1000
911/911 - 1s - loss: 0.0230 - val_loss: 0.0230
Epoch 60/1000
911/911 - 1s - loss: 0.0230 - val_loss: 0.0232
Epoch 61/1000
911/911 - 1s - loss: 0.0230 - val_loss: 0.0229
Epoch 62/1000
911/911 - 

...

Epoch 105/1000
911/911 - 1s - loss: 0.0700 - val_loss: 0.0704
Epoch 106/1000
911/911 - 1s - loss: 0.0700 - val_loss: 0.0707
Epoch 107/1000
911/911 - 1s - loss: 0.0699 - val_loss: 0.0705
Epoch 108/1000
911/911 - 1s - loss: 0.0699 - val_loss: 0.0702
Epoch 109/1000
911/911 - 1s - loss: 0.0699 - val_loss: 0.0710
Epoch 110/1000
911/911 - 1s - loss: 0.0698 - val_loss: 0.0702
Epoch 111/1000
911/911 - 1s - loss: 0.0698 - val_loss: 0.0704
Epoch 112/1000
911/911 - 1s - loss: 0.0698 - val_loss: 0.0703
Epoch 113/1000
911/911 - 1s - loss: 0.0698 - val_loss: 0.0702
Epoch 114/1000
911/911 - 1s - loss: 0.0697 - val_loss: 0.0704
Epoch 115/1000
911/911 - 1s - loss: 0.0697 - val_loss: 0.0702
Epoch 116/1000
911/911 - 1s - loss: 0.0697 - val_loss: 0.0701
Epoch 117/1000
911/911 - 1s - loss: 0.0696 - val_loss: 0.0704
Epoch 118/1000
911/911 - 1s - loss: 0.0696 - val_loss: 0.0701
Epoch 119/1000
911/911 - 1s - loss: 0.0696 - val_loss: 0.0700
Epoch 120/1000
911/911 - 1s - loss: 0.0696 - val_loss: 0.0699
Epoch 12

Epoch 118/1000
911/911 - 1s - loss: 0.1003 - val_loss: 0.1010
Epoch 119/1000
911/911 - 1s - loss: 0.1002 - val_loss: 0.1007
Epoch 120/1000
911/911 - 1s - loss: 0.1002 - val_loss: 0.1009
Epoch 121/1000
911/911 - 1s - loss: 0.1002 - val_loss: 0.1008
Epoch 122/1000
911/911 - 1s - loss: 0.1001 - val_loss: 0.1004
Epoch 123/1000
911/911 - 1s - loss: 0.1001 - val_loss: 0.1006
Epoch 124/1000
911/911 - 1s - loss: 0.1000 - val_loss: 0.1009
Epoch 125/1000
911/911 - 1s - loss: 0.1000 - val_loss: 0.1007
Epoch 126/1000
911/911 - 1s - loss: 0.1000 - val_loss: 0.1005
Epoch 127/1000
911/911 - 1s - loss: 0.1000 - val_loss: 0.1006
Epoch 128/1000
911/911 - 1s - loss: 0.0999 - val_loss: 0.1002
Epoch 129/1000
911/911 - 1s - loss: 0.0999 - val_loss: 0.1005
Epoch 130/1000
911/911 - 1s - loss: 0.0999 - val_loss: 0.1007
Epoch 131/1000
911/911 - 1s - loss: 0.0998 - val_loss: 0.1002
Epoch 132/1000
911/911 - 1s - loss: 0.0997 - val_loss: 0.1001
Epoch 133/1000
911/911 - 1s - loss: 0.0997 - val_loss: 0.1002
Epoch 13

Epoch 114/1000
911/911 - 1s - loss: 0.1556 - val_loss: 0.1569
Epoch 115/1000
911/911 - 1s - loss: 0.1555 - val_loss: 0.1561
Epoch 116/1000
911/911 - 1s - loss: 0.1555 - val_loss: 0.1560
Epoch 117/1000
911/911 - 1s - loss: 0.1555 - val_loss: 0.1559
Epoch 118/1000
911/911 - 1s - loss: 0.1554 - val_loss: 0.1562
Epoch 119/1000
911/911 - 1s - loss: 0.1553 - val_loss: 0.1571
Epoch 120/1000
911/911 - 1s - loss: 0.1553 - val_loss: 0.1592
Epoch 121/1000
911/911 - 1s - loss: 0.1552 - val_loss: 0.1560
Epoch 122/1000
911/911 - 1s - loss: 0.1553 - val_loss: 0.1556
Epoch 123/1000
911/911 - 1s - loss: 0.1552 - val_loss: 0.1562
Epoch 124/1000
911/911 - 1s - loss: 0.1551 - val_loss: 0.1554
Epoch 125/1000
911/911 - 1s - loss: 0.1551 - val_loss: 0.1560
Epoch 126/1000
911/911 - 1s - loss: 0.1550 - val_loss: 0.1560
Epoch 127/1000
911/911 - 1s - loss: 0.1550 - val_loss: 0.1557
Epoch 128/1000
911/911 - 1s - loss: 0.1549 - val_loss: 0.1560
Epoch 129/1000
911/911 - 1s - loss: 0.1549 - val_loss: 0.1557
Epoch 13

Epoch 112/1000
911/911 - 1s - loss: 0.2827 - val_loss: 0.2829
Epoch 113/1000
911/911 - 1s - loss: 0.2825 - val_loss: 0.2831
Epoch 114/1000
911/911 - 1s - loss: 0.2826 - val_loss: 0.2823
Epoch 115/1000
911/911 - 1s - loss: 0.2826 - val_loss: 0.2832
Epoch 116/1000
911/911 - 1s - loss: 0.2824 - val_loss: 0.2819
Epoch 117/1000
911/911 - 1s - loss: 0.2821 - val_loss: 0.2814
Epoch 118/1000
911/911 - 1s - loss: 0.2822 - val_loss: 0.2820
Epoch 119/1000
911/911 - 1s - loss: 0.2822 - val_loss: 0.2823
Epoch 120/1000
911/911 - 1s - loss: 0.2820 - val_loss: 0.2821
Epoch 121/1000
911/911 - 1s - loss: 0.2818 - val_loss: 0.2829
Epoch 122/1000
911/911 - 1s - loss: 0.2818 - val_loss: 0.2846
Epoch 123/1000
911/911 - 1s - loss: 0.2815 - val_loss: 0.2832
Epoch 124/1000
911/911 - 1s - loss: 0.2815 - val_loss: 0.2831
Epoch 125/1000
911/911 - 1s - loss: 0.2816 - val_loss: 0.2813
Epoch 126/1000
911/911 - 1s - loss: 0.2814 - val_loss: 0.2811
Epoch 127/1000
911/911 - 1s - loss: 0.2814 - val_loss: 0.2822
Epoch 12

911/911 - 1s - loss: 0.2733 - val_loss: 0.2745
Epoch 245/1000
911/911 - 1s - loss: 0.2733 - val_loss: 0.2737
Epoch 246/1000
911/911 - 1s - loss: 0.2730 - val_loss: 0.2734
Epoch 247/1000
911/911 - 1s - loss: 0.2731 - val_loss: 0.2723
Epoch 248/1000
911/911 - 1s - loss: 0.2732 - val_loss: 0.2737
Epoch 249/1000
911/911 - 1s - loss: 0.2730 - val_loss: 0.2743
Epoch 250/1000
911/911 - 1s - loss: 0.2730 - val_loss: 0.2737
Epoch 251/1000
911/911 - 1s - loss: 0.2729 - val_loss: 0.2724
Epoch 252/1000
911/911 - 1s - loss: 0.2727 - val_loss: 0.2727
Epoch 253/1000
911/911 - 1s - loss: 0.2730 - val_loss: 0.2720
Epoch 254/1000
911/911 - 1s - loss: 0.2728 - val_loss: 0.2727
Epoch 255/1000
911/911 - 1s - loss: 0.2729 - val_loss: 0.2726
Epoch 256/1000
911/911 - 1s - loss: 0.2725 - val_loss: 0.2737
Epoch 257/1000
911/911 - 1s - loss: 0.2727 - val_loss: 0.2736
Epoch 258/1000
911/911 - 1s - loss: 0.2726 - val_loss: 0.2727
Epoch 259/1000
911/911 - 1s - loss: 0.2726 - val_loss: 0.2728
Epoch 260/1000
911/911 

## 模型评估

In [69]:
# 评估函数定义
# 异常相关系数
import math
def acc(actual, predicted):
    pred_avg = np.average(predicted)

    act_avg = np.average(actual)
    diff_pred = predicted - pred_avg
    diff_act = actual - act_avg
    numerator = np.mean(np.sum(diff_pred*diff_act, axis=0))
    denominator = math.sqrt(np.mean(np.sum(diff_pred**2, axis=0)) * np.mean(np.sum(diff_act**2, axis=0)))
    ret_val = numerator/denominator
    return (100 * ret_val)


# 查看损失
def look_loss(history):
    plt.figure(figsize=(6.3, 2.5), dpi = 100)
    plt.plot(history.history['loss'], label='training data')
    plt.plot(history.history['val_loss'], label='validation data')
    plt.title('Loss')
    plt.ylabel('MSE Loss')
    plt.xlabel('Number of epochs')
    plt.grid()
    plt.legend(loc="upper right")
    plt.show()


# 模型预测
def all_estimate(best_model,x_test,y_test):
    testPred_1 = best_model.predict(x_test)
    a = y_test
    y_test_p = a.reshape(-1,1)
    testPred_p = testPred_1.reshape(-1,1)


    rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
    print('Test RMSE: %.3f' % rmse)
    #print('%.3f' % rmse)
    #print('acc:',acc(y_test_p,testPred_p))

    r2 = r2_score(y_test_p,testPred_p)
    print("R² score: %.4f" %r2)  
    #print("%.4f" % r2)
    

# def respective_estimate(best_model,x_test,y_test):
#     re2_list = []
#     rmse_list = []
#     for i in range(6):
#         # 模型预测
#         testPred = best_model.predict(x_test[i:i+1])
#         a = y_test[i:i+1]
#         y_test_p = a.reshape(-1,1)
#         testPred_p = testPred.reshape(-1,1)
#         r2 = r2_score(y_test_p,testPred_p)
#         rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
#         re2_list.append(r2)
#         rmse_list.append(rmse)    
#     return re2_list,rmse_list

In [70]:
#look_loss(history)

In [72]:
# 模型预测  
depth=[5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,1300,1500,1750,2000]
result = [] 
for i in range(0,33):
    #best_model = tf.keras.models.load_model('./model/BPNN/sali/bpnn_model_'+str(depth[i])+'m_2.h5') 
    best_model = tf.keras.models.load_model('D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/BPNN/temp/model1/bpnn_temp_'+str(depth[i])+'m.h5')
    print('预测第'+str(depth[i])+'m的温度')
    all_estimate(best_model,x_test,y_test[:,i:i+1])
    testPred = best_model.predict(x_test)
    result.append(testPred)

预测第5m的温度
Test RMSE: 0.034
R² score: 0.9988
预测第10m的温度
Test RMSE: 0.040
R² score: 0.9984
预测第15m的温度
Test RMSE: 0.047
R² score: 0.9977
预测第20m的温度
Test RMSE: 0.061
R² score: 0.9963
预测第30m的温度
Test RMSE: 0.093
R² score: 0.9912
预测第40m的温度
Test RMSE: 0.124
R² score: 0.9844
预测第50m的温度
Test RMSE: 0.147
R² score: 0.9778
预测第60m的温度
Test RMSE: 0.169
R² score: 0.9700
预测第70m的温度
Test RMSE: 0.181
R² score: 0.9643
预测第80m的温度
Test RMSE: 0.196
R² score: 0.9571
预测第90m的温度
Test RMSE: 0.202
R² score: 0.9536
预测第100m的温度
Test RMSE: 0.210
R² score: 0.9490
预测第125m的温度
Test RMSE: 0.219
R² score: 0.9432
预测第150m的温度
Test RMSE: 0.220
R² score: 0.9420
预测第175m的温度
Test RMSE: 0.216
R² score: 0.9453
预测第200m的温度
Test RMSE: 0.216
R² score: 0.9466
预测第225m的温度
Test RMSE: 0.215
R² score: 0.9485
预测第250m的温度
Test RMSE: 0.209
R² score: 0.9525
预测第275m的温度
Test RMSE: 0.209
R² score: 0.9537
预测第300m的温度
Test RMSE: 0.211
R² score: 0.9534
预测第350m的温度
Test RMSE: 0.216
R² score: 0.9522
预测第400m的温度
Test RMSE: 0.230
R² score: 0.9467
预测第500m的温度
Test RMSE: 

(33, 1242592)

In [148]:
best_model = tf.keras.models.load_model('D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/BPNN/temp/model1/bpnn_temp_2000m.h5')
testPred = best_model.predict(x_test)   

# 预测结果可视化

## 为归一化之后的预测值填充nan

In [145]:
# 查看原始数据的形状
#test_flatten.shape 

(2016000, 38)

In [147]:
#depth=[5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,1300,1500,1750,2000]
for i in range(0,33):
    best_model = tf.keras.models.load_model('D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/BPNN/temp/model1/bpnn_temp_'+str(depth[i])+'m.h5')
    testPred = best_model.predict(x_test)  
    nan_mask = np.isnan(test_flatten) #返回一个与 test_flatten 形状相同的布尔数组，其中对应的元素是 NaN 时为 True，否则为 False
    nan_masks = nan_mask.any(axis=1) # true 代表通道中有nan   
    
    testPred_with_nan = np.full(((nan_masks.shape[0]),), np.nan)  # 创建一个形状与nan_masks相同且全是nan的数组
    num = 0 
    for j,nan_mask in enumerate(nan_masks):   # i从0开始

        if (nan_mask == False): # False 代表没有nan，使用预测值进行填充
            testPred_with_nan[j] = testPred[num]
            num = num + 1
        
    print(num)

1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592
1242592


In [149]:
# 查看预测结果（testPred）与原始数据的形状
testPred.shape,test_flatten.shape 

((1242592, 1), (2016000, 38))

In [150]:
nan_mask = np.isnan(test_flatten) #返回一个与 test_flatten 形状相同的布尔数组，其中对应的元素是 NaN 时为 True，否则为 False
nan_masks = nan_mask.any(axis=1) # true 代表通道中有nan   
print(nan_masks.shape)

(2016000,)


In [151]:
### 创建一个形状与nan_masks相同且全是nan的数组
testPred_with_nan = np.full(((nan_masks.shape[0]),), np.nan)  

In [152]:
num = 0 
for i,nan_mask in enumerate(nan_masks):   # i从0开始

    if (nan_mask == False): # False 代表没有nan，使用预测值进行填充
        testPred_with_nan[i] = testPred[num]
        num = num + 1
        
print(num)

1242592


## 为归一化之后的真实值填充nan 

In [132]:
print(nan_masks.shape)

(2016000,)


In [133]:
y_test_with_nan = np.full(((nan_masks.shape[0]),), np.nan) 

In [134]:
y_test[:,0:33].shape 

(1242592, 33)

In [154]:
num = 0 
for i,nan_mask in enumerate(nan_masks):   # i从0开始

    if (nan_mask == False): # False 代表没有nan，使用预测值进行填充
        y_test_with_nan[i] = y_test[:,32:33][num]  # 14代表第14层，应根据具体的层数进行替换。
        num = num + 1
        
print(num)
#for j in range(33):
#    num = 0 
#    for i,nan_mask in enumerate(nan_masks):   # i从0开始
#        if (nan_mask == False): # False 代表没有nan，使用预测值进行填充
#            y_test_with_nan[i] = y_test[:,j:j+1][num]  # 14代表第14层，应根据具体的层数进行替换。
#            num = num + 1
#    print(num)
#j = j + 1

1242592


# 结果可视化

In [155]:
testPred_with_nan_reshape  = testPred_with_nan.reshape((12,280,600))
y_test_with_nan_reshape = y_test_with_nan.reshape((12,280,600))
testPred_with_nan_reshape.shape , y_test_with_nan_reshape.shape 

((12, 280, 600), (12, 280, 600))

In [156]:
lat = []
lon = []
time = []

In [157]:
lon.append(data_ssh['lon'][:].data)
lat.append(data_ssh['lat'][:].data)

In [158]:
start_year = 2015
end_year =2015
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [159]:
new_NC = nc.Dataset("D:/codeFile/jupyterDemo/sj/BPNN/sst/result2000m.nc", 'w', format='NETCDF4')

'''
定义维度，后一个参数表示维度的长度，因为是合并的同一个产品的数据，所以是统一
的,注意维度的长度一定要和读入的数据匹配
'''

new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))
new_NC.createDimension('true_sta', len(y_test_with_nan_reshape))
new_NC.createDimension('pred_sta', len(testPred_with_nan_reshape))

#new_NC.createDimension('level', 20)
new_NC.createDimension('time', len(time))




new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
#new_NC.createVariable('level', 'f', ("level"))
new_NC.createVariable('true_sta', 'f',("time","lat","lon"))
new_NC.createVariable('pred_sta', 'f',("time","lat","lon"))

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2015-01-15'
time_var.long_name = 'Months in Monthly Means'
time_var.axis = 'T'



#向变量中填充数据
new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]
new_NC.variables['time'][:] = np.array(time)
#new_NC.variables['level'][:] = level

new_NC.variables['true_sta'][:]=np.array(y_test_with_nan_reshape)
new_NC.variables['pred_sta'][:]=np.array(testPred_with_nan_reshape)


#最后记得关闭文件
new_NC.close()


In [160]:
y_test_with_nan_reshape.shape 

(12, 280, 600)